In [2]:
import argparse
import functools
import gc
import re
import logging
import math
import os
import random
import shutil
from contextlib import nullcontext
from pathlib import Path

import accelerate
import numpy as np
import torch
import torch.nn.functional as F
import torch.utils.checkpoint
import transformers

from omegaconf import OmegaConf
from functools import partial
from accelerate import Accelerator
from accelerate.logging import get_logger
from accelerate.utils import DistributedType, ProjectConfiguration, set_seed
from datasets import load_dataset
from huggingface_hub import create_repo, upload_folder
from packaging import version
from PIL import Image
from torchvision import transforms
from tqdm.auto import tqdm
from transformers import AutoTokenizer, PretrainedConfig

import diffusers
from diffusers import (
    AutoencoderKL,
    DDPMScheduler,
    UniPCMultistepScheduler,
)
from utils.utils import (
    exists,
    instantiate_from_config
)

In [7]:
from diffusers import AutoencoderKL
from PIL import Image
import  torch
import torchvision.transforms as T

cfg = OmegaConf.load("config/srganResNet.yaml")
srganresnet  = instantiate_from_config(cfg.model.srganresnet)
loadnet = torch.load(cfg.train.noiseresnet_path, map_location="cpu")
srganresnet.load_state_dict(loadnet, strict=True)


            
noisestep=100
#  ./huggingface/stable-diffusion-v1-4/vae 切换为任意本地路径
vae = AutoencoderKL.from_pretrained("checkpoint/sdxl-vae-fp16-fix")
noise_scheduler = DDPMScheduler.from_pretrained("checkpoint/stable-diffusion-xl-refiner-1.0", subfolder="scheduler")
#vae.to("cuda")
def encode_img(input_img):
    transform = T.Compose([
        T.ToTensor()
    ])
    input_img = transform(input_img)#.to("cuda")
    if len(input_img.shape)<4:
        input_img = input_img.unsqueeze(0)
        # input_img = F.interpolate(
        #             input_img,
        #             scale_factor=4,  # 或直接指定尺寸 mode='bilinear', align_corners=False
        #             mode='bicubic'  # 可选 'nearest', 'bicubic' 等
        #         )
        input_img = input_img.clamp(0, 1) * 2 - 1
    with torch.no_grad():
        latents = vae.encode(input_img)
        latents = latents.latent_dist.sample()* vae.config.scaling_factor # Note scaling

    return latents


def decode_img(latents):
    # bath of latents -> list of images
    latents = (1 / vae.config.scaling_factor) * latents
    print(latents.device)
    with torch.no_grad():
        image = vae.decode(latents).sample
    image = (image / 2 + 0.5).clamp(0, 1)
    image = image.detach().cpu()
    # image = T.Resize(original_size)(image.squeeze())
    return T.ToPILImage()(image.squeeze())

if __name__ == '__main__':
    # Load an example image
    input_img = Image.open("AIDtest_pro/HR/pond_293_hr.png")
    
    # Encode and decode the image
    latents = encode_img(input_img)
    #latents = srganresnet(latents)
    reconstructed_img = decode_img(latents)

    # Save the reconstructed image
    reconstructed_img.save("reconstructed3.png")
    # Concatenate the original and reconstructed images
    # concatenated_img = Image.new('RGB', (original_size[0] * 2, original_size[1]))
    # concatenated_img.paste(input_img, (0, 0))
    # concatenated_img.paste(reconstructed_img, (original_size[0], 0))
    # # Save the concatenated image
    # concatenated_img.save("vaetest/concatenated_church_136.png")


/tmp/ipykernel_1280/153793454.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loadnet = torch.load(cfg.train.noiseresnet_path, map_location="cpu")


cpu


In [3]:
from diffusers import AutoencoderKL
from PIL import Image
import  torch
import torchvision.transforms as T

cfg = OmegaConf.load("config/srganResNet.yaml")
srganresnet  = instantiate_from_config(cfg.model.srganresnet)
loadnet = torch.load(cfg.train.noiseresnet_path, map_location="cpu")
srganresnet.load_state_dict(loadnet, strict=True)

tensor_4x4x64 = torch.ones((4, 5, 5))
tensor_4x4x64 = srganresnet(tensor_4x4x64)
print(tensor_4x4x64)

tensor([[[0.0797, 0.1735, 0.3032, 0.2582, 0.1050],
         [0.2624, 0.3757, 0.4058, 0.3323, 0.1815],
         [0.2848, 0.3722, 0.3713, 0.3961, 0.3012],
         [0.3829, 0.5861, 0.5681, 0.5723, 0.3273],
         [0.2770, 0.3557, 0.3824, 0.3813, 0.2155]],

        [[0.3474, 0.4726, 0.5803, 0.4147, 0.1840],
         [0.2612, 0.5787, 0.7942, 0.5793, 0.1970],
         [0.3089, 0.6913, 0.9542, 0.7457, 0.2591],
         [0.1641, 0.4254, 0.6405, 0.5139, 0.2148],
         [0.1081, 0.2023, 0.3134, 0.2891, 0.1031]],

        [[0.4995, 0.9712, 1.0125, 0.8367, 0.4393],
         [0.7838, 1.4252, 1.4664, 1.2186, 0.6746],
         [0.8107, 1.5177, 1.5462, 1.3267, 0.6857],
         [0.7062, 1.3419, 1.3833, 1.1374, 0.6056],
         [0.2482, 0.5513, 0.5762, 0.5304, 0.2977]],

        [[0.1522, 0.2745, 0.3789, 0.3018, 0.2313],
         [0.2767, 0.4106, 0.5960, 0.4656, 0.2551],
         [0.3060, 0.4846, 0.6594, 0.4780, 0.2072],
         [0.3041, 0.4796, 0.5958, 0.4444, 0.2172],
         [0.2908, 0.3787,

/tmp/ipykernel_1250/972906042.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loadnet = torch.load(cfg.train.noiseresnet_path, map_location="cpu")


In [1]:
import torch

# 生成标准正态分布的tensor (μ=0, σ=1)
normal_tensor = torch.randn(4, 64, 64)
print(normal_tensor.shape)  # 输出: torch.Size([4, 64, 64])
print(f"均值: {normal_tensor.mean():.4f}, 标准差: {normal_tensor.std():.4f}")

torch.Size([4, 64, 64])
均值: 0.0051, 标准差: 0.9968


In [4]:
import numpy as np
import cv2

# 读取灰度图
gray_image = cv2.imread('train_maskmodel/example/diff/diff-200/img_0.png', cv2.IMREAD_GRAYSCALE)

# 二值化处理
threshold = 255 * 0.3
binary_image = np.where(gray_image > threshold, 1, 0).astype(np.uint8)

# 保存结果
cv2.imwrite('binary_output.png', binary_image * 255)  # 保存时需将1映射为255（白色）

True